In [99]:
import io
import pandas as pd
import requests
import re
import xml.etree.ElementTree as ET
import subprocess

from bs4 import BeautifulSoup

In [17]:
tree = ET.parse('ABSA16FR_Restaurants_Train.xml')
root = tree.getroot()

In [5]:
links = list()

In [9]:
f = open("ABSA16FR_Restaurants_index.txt", "r")
for x in f:
    val = x.split('\t')
    val[2] = val[2].replace('\n', '')
    links.append(val)

In [35]:
dataL = pd.DataFrame(links, columns=['id', 'link', 'nbSentence'])

In [36]:
dataL = dataL.set_index('id')
dataL.head()

,link,nbSentence
id,,
g1079435-d3498474-r271346275,https://www.tripadvisor.fr/ShowUserReviews-g10...,1
g1080298-d1332050-r275655485,https://www.tripadvisor.fr/ShowUserReviews-g10...,2
g1080298-d4701165-r281527562,https://www.tripadvisor.fr/ShowUserReviews-g10...,3
g1080559-d3686505-r151173923,https://www.tripadvisor.fr/ShowUserReviews-g10...,1
g1080574-d1337519-r211045323,https://www.tripadvisor.fr/ShowUserReviews-g10...,6


In [40]:
dataL.index

Index(['g1079435-d3498474-r271346275', 'g1080298-d1332050-r275655485',
       'g1080298-d4701165-r281527562', 'g1080559-d3686505-r151173923',
       'g1080574-d1337519-r211045323', 'g1582538-d3877732-r250915172',
       'g1833139-d7211477-r282922737', 'g187073-d1328515-r282975754',
       'g187073-d2234111-r233672137', 'g187073-d2346663-r285268023',
       ...
       'g3463365-d824605-r268214202', 'g608761-d1334387-r277341177',
       'g608761-d1334432-r281522672', 'g608761-d2053817-r143008138',
       'g608761-d2445738-r244940036', 'g608761-d3778492-r181222052',
       'g608761-d4750470-r279534176', 'g608761-d5960507-r277194025',
       'g608761-d7105002-r282300173', 'g635821-d5792203-r217535741'],
      dtype='object', name='id', length=337)

In [44]:
r = requests.get(dataL.loc['g1079435-d3498474-r271346275']['link'])

In [45]:
m = re.search('<p class="partial_entry">(.)*[a-zA-Z0-9]<\/p>', r.text)
if m:
    found = m.group(0)

In [58]:
len('<p class="partial_entry">')
s = "abcdefghijklmnopq"
s[5:-2]

'fghijklmno'

In [117]:
data = list()
dataOpinion = list()

In [150]:
l = len(root)
i = 0
pa = 0
notfound = 0
for review in root:
    id = review.attrib['rid']
    r = requests.get(dataL.loc[id]['link'])
    doc = BeautifulSoup(r.text)
    doc = doc.find('p', attrs={'class' : 'partial_entry'})
    if doc != None:
        doc = doc.get_text()
   
        
        with open("D:\Git\TER-Sentiment-Analysis\data\input.txt", "w", encoding="utf-8") as f: # or "rb"
            f.write(doc)
            f.close()
        subprocess.call("D:\\Git\\TER-Sentiment-Analysis\\data\\apache-opennlp-1.9.2\\bin\\opennlp.bat SentenceDetector D:\\Git\\TER-Sentiment-Analysis\\data\\apache-opennlp-1.9.2\\models\\en-sent.bin < D:\\Git\\TER-Sentiment-Analysis\\data\\input.txt > D:\\Git\\TER-Sentiment-Analysis\\data\\sentences.txt") 
        with open('D:\Git\TER-Sentiment-Analysis\data\sentences.txt', 'r') as content_file:
            content = content_file.read()
            index_sentence = 0
            sentences = content.split('\n')[:-2]
            for sentence in sentences:
                #print(len(review[0]))
                #print("========")
                #print(sentences)
                if len(review[0]) == len(sentences):
                    if len(review[0][index_sentence]) > 0:
                        opinions = review[0][index_sentence][1]
                        for opinion in opinions:
                            data.append([id, str(index_sentence), sentence, opinion.attrib['target'], opinion.attrib['category'], opinion.attrib['polarity'], opinion.attrib['from'], opinion.attrib['to']])
                    else:
                        data.append([id, str(index_sentence), sentence, None, None, None, None, None])
                else:
                    print(str(i)+"/"+str(l)+" "+id+" Pass", end="\r")
                    pa += 1
                    continue
                index_sentence += 1
        print(str(i)+"/"+str(l)+" "+id, end="\r")
        
    else:
        notfound += 1
        print("Ressource {} doesn't contains a review".format(id))
    
    i += 1     

Ressource g1080574-d1337519-r211045323 doesn't contains a review
Ressource g187073-d1328515-r282975754 doesn't contains a review
Ressource g187073-d2234111-r233672137 doesn't contains a review
Ressource g187091-d2221132-r263477843 doesn't contains a review
Ressource g187147-d1113739-r231078339 doesn't contains a review
Ressource g187147-d1650840-r242497721 doesn't contains a review
Ressource g187147-d2087172-r264852052 doesn't contains a review
Ressource g187147-d4235143-r220435098 doesn't contains a review
Ressource g187147-d4272002-r260711186 doesn't contains a review
Ressource g187147-d6877828-r267314039 doesn't contains a review
Ressource g187147-d718132-r119263750 doesn't contains a review
Ressource g187147-d804109-r161366715 doesn't contains a review
Ressource g187147-d947484-r266104321 doesn't contains a review
Ressource g187198-d2041733-r222676078 doesn't contains a review
Ressource g187253-d1223545-r227663690 doesn't contains a review
Ressource g196680-d7702811-r279218028 does

In [151]:
print("Ressources not found (404) : "+str(notfound))
print("Sentence pass due to SentenceDetector : "+str(pa))
print("Total : "+str(337-notfound))

Ressources not found (404) : 18
Sentence pass due to SentenceDetector : 599
Total : 319


In [148]:
dataDf = pd.DataFrame(data, columns=['id', 'index_sentence', 'sentence', 'target', 'category', 'polarity', 'from', 'to'])

In [149]:
dataDf

,id,index_sentence,sentence,target,category,polarity,from,to
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
dataDf.to_csv (r'D:\Git\TER-Sentiment-Analysis\data\export_data.csv', header=True)